In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")
# application_df.head()
application_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30353 entries, 0 to 30352
Data columns (total 12 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   EIN                     30353 non-null  int64  
 1   NAME                    30353 non-null  object 
 2   APPLICATION_TYPE        30353 non-null  object 
 3   AFFILIATION             30353 non-null  object 
 4   CLASSIFICATION          30353 non-null  object 
 5   USE_CASE                30353 non-null  object 
 6   ORGANIZATION            30352 non-null  object 
 7   STATUS                  30352 non-null  float64
 8   INCOME_AMT              30352 non-null  object 
 9   SPECIAL_CONSIDERATIONS  30352 non-null  object 
 10  ASK_AMT                 30352 non-null  float64
 11  IS_SUCCESSFUL           30352 non-null  float64
dtypes: float64(3), int64(1), object(8)
memory usage: 2.8+ MB


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
#  YOUR CODE GOES HERE
application_df = application_df.drop(["EIN","NAME"],1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
# Determine the number of unique values in each column.
#  YOUR CODE GOES HERE
application_df.nunique()


EIN                       30353
NAME                      17676
APPLICATION_TYPE             15
AFFILIATION                   6
CLASSIFICATION               69
USE_CASE                      6
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    7463
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
#  YOUR CODE GOES HERE
apptype_count=application_df.APPLICATION_TYPE.value_counts()
apptype_count

T3     24164
T4      1256
T6      1089
T5       958
T19      891
T8       638
T7       630
T10      485
T9       138
T13       64
T12       20
T2        13
T14        3
T29        2
T25        2
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
app_types_to_replace = list(apptype_count[apptype_count < 500].index)

# Replace in dataframe
for app in app_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       24164
T4        1256
T6        1089
T5         958
T19        891
Other      727
T8         638
T7         630
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_cnt=application_df.CLASSIFICATION.value_counts()
print(classification_cnt)
print("-"*50)
print(f"Average: {classification_cnt.mean()}")
print(f"Median: {classification_cnt.median()}") 

C1000    15164
C2000     5294
C1200     4455
C2100     1833
C3000     1507
         ...  
C2170        1
C4120        1
C8210        1
C4500        1
C1820        1
Name: CLASSIFICATION, Length: 69, dtype: int64
--------------------------------------------------
Average: 439.8985507246377
Median: 6.0


In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1

cls_to_remove = pd.DataFrame(classification_cnt.loc[lambda x : x == 1])
cls_to_remove_list = cls_to_remove.index.tolist()

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced

class_to_replace = list(classification_cnt[classification_cnt < 500 ].index)

# Replace in dataframe
for cls in class_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    15164
C2000     5294
C1200     4455
C2100     1833
C3000     1507
Other     1388
C7000      712
Name: CLASSIFICATION, dtype: int64

In [24]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df,
                                        columns = ["APPLICATION_TYPE", 
                                                   "AFFILIATION", 
                                                   "CLASSIFICATION", 
                                                   "USE_CASE", 
                                                   "ORGANIZATION", 
                                                   "INCOME_AMT", 
                                                   "SPECIAL_CONSIDERATIONS"], 
                                        drop_first = True)


In [22]:
application_df_dummies.head()

,EIN,NAME,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_Y
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,1.0,5000.0,1.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,1.0,108590.0,1.0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,1.0,5000.0,0.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,1.0,6692.0,1.0,0,1,0,0,0,...,1,0,1,0,0,0,0,0,0,0
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,1.0,142590.0,1.0,0,1,0,0,0,...,1,0,0,1,0,0,0,0,0,0


In [28]:
# Split our preprocessed data into our features and target arrays
y = application_df_dummies.IS_SUCCESSFUL
X = application_df_dummies.drop(columns="IS_SUCCESSFUL")
X = X.drop(columns= "NAME")

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [29]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model